In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import logging
import random
import pkg_resources
import sklearn
from sklearn.model_selection import train_test_split
from rxnfp.tokenization import *
# from rdkit import Chem
# from rdkit.Chem import rdChemReactions
torch.cuda.is_available()
import rxnfp
#from rxnfp.models import SmilesClassificationModel
#from rxn_yields.core import SmilesTokenizer, SmilesClassificationModel

logger = logging.getLogger(__name__)
torch.cuda.is_available()
# from simpletransformers.classification import ClassificationModel, ClassificationArgs
from rxnfp.tokenization import SmilesTokenizer

/opt/conda/envs/reaction-workbench-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from rxnfp.tokenization import SmilesTokenizer

import os
import pickle

import argparse

from tqdm.auto import tqdm
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix

In [3]:
train = pd.read_csv('../ReactionClassification_2024/data/train.csv', delimiter=',')

In [4]:
val = pd.read_csv('../ReactionClassification_2024/data/val.csv', delimiter=',')

In [5]:
test = pd.read_csv('../ReactionClassification_2024/data/test.csv', delimiter=',')

In [6]:
train1 = pd.read_csv('../ReactionClassification_2024/data/train1.csv', delimiter=',')
train2 = pd.read_csv('../ReactionClassification_2024/data/train2.csv', delimiter=',')
train3 = pd.read_csv('../ReactionClassification_2024/data/train3.csv', delimiter=',')
train4 = pd.read_csv('../ReactionClassification_2024/data/train4.csv', delimiter=',')
train5 = pd.read_csv('../ReactionClassification_2024/data/train5.csv', delimiter=',')
train6 = pd.read_csv('../ReactionClassification_2024/data/train6.csv', delimiter=',')
train7 = pd.read_csv('../ReactionClassification_2024/data/train7.csv', delimiter=',')
train8 = pd.read_csv('../ReactionClassification_2024/data/train8.csv', delimiter=',')
train9 = pd.read_csv('../ReactionClassification_2024/data/train9.csv', delimiter=',')
train10 = pd.read_csv('../ReactionClassification_2024/data/train10.csv', delimiter=',')

### Train Embedding

In [14]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return(sum_embeddings / sum_mask)

In [15]:
model_path = 'data/fine_tuned_model_Final2'
#model_path = 'rxn-data-from-postgresql/models/reaxys_bert/checkpoint-4356432-epoch-36'

model = AutoModel.from_pretrained(model_path, output_hidden_states=True)
#tokenizer = SmilesTokenizer(vocab_file='rxn-data-from-postgresql/models/reaxys_bert/checkpoint-4235420-epoch-35/vocab.txt')
tokenizer = SmilesTokenizer(vocab_file='data/fine_tuned_model_Final2/vocab.txt')

In [17]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
val_sentence_embeddings = []
test_sentence_embeddings = []
rare_sentence_embeddings = []
batch= 64

In [12]:
for idx in range(1, 11):
    print(idx)
    train_df = globals()[f'train{idx}']
    train_reactions = train_df['reaction'].astype(str).tolist()
    sentence_embeddings = []
    batch_size = 64  # Define a suitable batch size

    for i in tqdm(range(0, len(train_reactions), batch_size), desc=f'Processing train{idx}'):
        # Tokenize sentences
        encoded_input = tokenizer(train_reactions[i:i+batch_size], padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)

        # Compute token embeddings
        with torch.no_grad():
            model_output = model(**encoded_input)

        # Perform pooling
        sentence_embeddings.extend([x.cpu().detach().numpy() for x in mean_pooling(model_output, encoded_input['attention_mask'])])

    # Assign embeddings to the DataFrame and save to files
    train_df['OptBertFP'] = sentence_embeddings
    X_train = np.array(train_df['OptBertFP'])
    np.save(f'../ReactionClassification_2024/data/OptBertFPFinal2/X_train{idx}_OptBertFP.npy', X_train)
    #train_df.to_csv(f'../ReactionClassification_2024/data/PreBertFP/train{idx}_PreBertFP.csv', index=False)

1


Processing train1: 100%|██████████| 3759/3759 [02:49<00:00, 22.16it/s]


2


Processing train3: 100%|██████████| 3759/3759 [02:21<00:00, 26.63it/s]


4


...

Processing train8: 100%|██████████| 3759/3759 [03:00<00:00, 20.79it/s]


9


Processing train9: 100%|██████████| 3759/3759 [03:00<00:00, 20.87it/s]


10


Processing train10: 100%|██████████| 3759/3759 [02:53<00:00, 21.64it/s]


In [ ]:
#train.to_csv('data/PreBertFP/train_PreBertFP.csv', index=False)

### Test and Val Embedding

In [13]:
val_reactions = val['reaction'].astype(str).tolist()
for i in tqdm(range(0, len(val_reactions), batch)):
    #Tokenize sentences
    encoded_input = tokenizer(val_reactions[i:i+batch], padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)

    #Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    #Perform pooling. In this case, mean pooling
    val_sentence_embeddings.extend([x.cpu().detach().numpy() for x in mean_pooling(model_output, encoded_input['attention_mask'])])

100%|██████████| 5175/5175 [04:10<00:00, 20.68it/s]


In [14]:
val['OptBertFP'] = val_sentence_embeddings

In [15]:
X_val = np.array(val['OptBertFP'])

In [16]:
np.save('../ReactionClassification_2024/data/OptBertFPFinal2/X_val_OptBertFP.npy', X_val)

In [17]:
#val.to_csv('data/PreBertFP/val_PreBertFP.csv', index=False)

In [18]:
test_reactions = test['reaction'].astype(str).tolist()
for i in tqdm(range(0, len(test_reactions), batch)):
    #Tokenize sentences
    encoded_input = tokenizer(test_reactions[i:i+batch], padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)

    #Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    #Perform pooling. In this case, mean pooling
    test_sentence_embeddings.extend([x.cpu().detach().numpy() for x in mean_pooling(model_output, encoded_input['attention_mask'])])

100%|██████████| 5169/5169 [04:14<00:00, 20.33it/s]


In [19]:
test['OptBertFP'] = test_sentence_embeddings
X_test = np.array(test['OptBertFP'])

In [20]:
np.save('../ReactionClassification_2024/data/OptBertFPFinal2/X_test_OptBertFP.npy', X_test)

In [17]:
#test.to_csv('data/PreBertFP/test_PreBertFP.csv', index=False)

### Data Load

In [3]:
X_train1 = np.load('../ReactionClassification_2024/data/OptBertFPFinal2/X_train1_OptBertFP.npy',allow_pickle=True)
X_train2 = np.load('../ReactionClassification_2024/data/OptBertFPFinal2/X_train2_OptBertFP.npy',allow_pickle=True)
X_train3 = np.load('../ReactionClassification_2024/data/OptBertFPFinal2/X_train3_OptBertFP.npy',allow_pickle=True)
X_train4 = np.load('../ReactionClassification_2024/data/OptBertFPFinal2/X_train4_OptBertFP.npy',allow_pickle=True)
X_train5 = np.load('../ReactionClassification_2024/data/OptBertFPFinal2/X_train5_OptBertFP.npy',allow_pickle=True)
X_train6 = np.load('../ReactionClassification_2024/data/OptBertFPFinal2/X_train6_OptBertFP.npy',allow_pickle=True)
X_train7 = np.load('../ReactionClassification_2024/data/OptBertFPFinal2/X_train7_OptBertFP.npy',allow_pickle=True)
X_train8 = np.load('../ReactionClassification_2024/data/OptBertFPFinal2/X_train8_OptBertFP.npy',allow_pickle=True)
X_train9 = np.load('../ReactionClassification_2024/data/OptBertFPFinal2/X_train9_OptBertFP.npy',allow_pickle=True)
X_train10 = np.load('../ReactionClassification_2024/data/OptBertFPFinal2/X_train10_OptBertFP.npy',allow_pickle=True)

In [4]:
X_train = np.concatenate([X_train1, X_train2, X_train3, X_train4, X_train5,
                          X_train6, X_train7, X_train8, X_train9, X_train10], axis=0)

In [5]:
X_train1 = np.vstack(X_train)
X_train = np.squeeze(X_train1)

In [6]:
#X_train = np.load('data/RXNFP/X_train_RXNFP.npy')
X_test = np.load('../ReactionClassification_2024/data/OptBertFPFinal2/X_test_OptBertFP.npy',allow_pickle=True)
X_val = np.load('../ReactionClassification_2024/data/OptBertFPFinal2/X_val_OptBertFP.npy',allow_pickle=True)

In [7]:
X_test1 = np.vstack(X_test)
X_test = np.squeeze(X_test1)

In [8]:
X_val1 = np.vstack(X_val)
X_val = np.squeeze(X_val1)

In [9]:
train = pd.read_csv('../ReactionClassification_2024/data/train.csv', delimiter=',')
test = pd.read_csv('../ReactionClassification_2024/data/test.csv', delimiter=',')
val = pd.read_csv('../ReactionClassification_2024/data/val.csv', delimiter=',')

In [21]:
y_train = train['CLASS-ID'].values
y_test = test['CLASS-ID'].values
y_val = val['CLASS-ID'].values
y_rare = rare['CLASS-ID'].values

In [13]:
rare = pd.read_csv('../ReactionClassification_2024/data/rare.csv', delimiter=',')

In [18]:
rare_reactions = rare['reaction'].astype(str).tolist()
for i in tqdm(range(0, len(rare_reactions), batch)):
    #Tokenize sentences
    encoded_input = tokenizer(rare_reactions[i:i+batch], padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)

    #Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    #Perform pooling. In this case, mean pooling
    rare_sentence_embeddings.extend([x.cpu().detach().numpy() for x in mean_pooling(model_output, encoded_input['attention_mask'])])

100%|██████████| 13/13 [00:02<00:00,  5.43it/s]


In [19]:
rare['OptBertFP'] = rare_sentence_embeddings
X_rare = np.array(rare['OptBertFP'])

In [20]:
np.save('../ReactionClassification_2024/data/OptBertFPFinal2/X_rare_OptBertFP.npy', X_rare)

### Test on Val set

In [22]:
import faiss
index = faiss.IndexFlatL2(X_train.shape[1])
index.add(X_train)

In [12]:
from tqdm import tqdm
k = 1 

batch_size = 1000

num_batches = (X_val.shape[0] + batch_size - 1) // batch_size

In [13]:
D_val_1nn = np.zeros((X_val.shape[0], k), dtype=np.float32)
I_val_1nn = np.zeros((X_val.shape[0], k), dtype=np.int64)

for b in tqdm(range(num_batches), desc='FAISS Search on Val Data'):
    start = b * batch_size
    end = min((b + 1) * batch_size, X_val.shape[0])
    D, I = index.search(X_val[start:end], k)
    D_val_1nn[start:end, :] = D
    I_val_1nn[start:end, :] = I

FAISS Search on Val Data: 100%|██████████| 332/332 [1:59:34<00:00, 21.61s/it]  


In [14]:
y_pred_val_1nn = np.array(y_train[I_val_1nn])

In [21]:
np.save('../ReactionClassification_2024/data/OptBertFPFinal2/y_pred_val_1nn', y_pred_val_1nn)

In [22]:
y_pred_val_1nn = np.load('../ReactionClassification_2024/data/OptBertFPFinal2/y_pred_val_1nn.npy')

In [23]:
print(f'Overall Accuracy: {accuracy_score(y_val, y_pred_val_1nn)}')

Overall Accuracy: 0.9612749523674365


### Test on Test set

In [24]:
k = 1  
batch_size = 1000  

num_batches = (X_test.shape[0] + batch_size - 1) // batch_size

In [ ]:
D_test_1nn = np.zeros((X_test.shape[0], k), dtype=np.float32)
I_test_1nn = np.zeros((X_test.shape[0], k), dtype=np.int64)

for b in tqdm(range(num_batches), desc='FAISS Search on Test Data'):
    start = b * batch_size
    end = min((b + 1) * batch_size, X_test.shape[0])
    D, I = index.search(X_test[start:end], k)
    D_test_1nn[start:end, :] = D
    I_test_1nn[start:end, :] = I

FAISS Search on Test Data:  73%|███████▎  | 240/331 [1:26:24<32:26, 21.39s/it]

In [ ]:
y_pred_test_1nn = np.array(y_train[I_test_1nn])

In [ ]:
np.save('../ReactionClassification_2024/data/OptBertFPFinal2/y_pred_test_1nn', y_pred_test_1nn)

In [ ]:
y_pred_test_1nn = np.load('../ReactionClassification_2024/data/OptBertFPFinal2/y_pred_test_1nn.npy')

In [ ]:
print(f'Overall Accuracy: {accuracy_score(y_test, y_pred_test_1nn)}')

In [ ]:
report_test1nn = classification_report(y_test, y_pred_test_1nn, output_dict=True)
report_df = pd.DataFrame(report_test1nn).transpose()
df_test1nn = report_df[:-3].reset_index().rename(columns={'index': 'CLASS-ID'})
df_test1nn['test_support'] = df_test1nn['support'].astype(int)
df_test1nn['CLASS-ID'] = df_test1nn['CLASS-ID'].astype(int)

In [ ]:
train_class_support = train['CLASS-ID'].value_counts().sort_index()
train_support_df = train_class_support.reset_index()
train_support_df.columns = ['CLASS-ID', 'train_support']
train_support_df['CLASS-ID'] = train_support_df['CLASS-ID'].astype(int)

In [ ]:
# Merge with df_val1nn on 'CLASS-ID'
df_test1nn_report = pd.merge(df_test1nn, train_support_df, on='CLASS-ID', how='left')
# Sort based on the number of train_support
df_test1nn_report = df_test1nn_report.sort_values(by='train_support', ascending=False)
df_test1nn_report = df_test1nn_report.drop(columns=['support'])

In [ ]:
df_class = pd.read_csv('../ReactionClassification_2024/data/className.tsv', sep='\t', encoding='ISO-8859-1')

In [ ]:
# Convert multi Class-ID into one
def clean_class_id(row):
    # Split the string by comma and convert to a list
    class_ids = str(row['CLASS-ID']).split(',')
    # Return the first element from the list, ensuring it's an integer
    return int(class_ids[0].strip())

# Apply the function to the 'CLASS-ID' column
df_class['CLASS-ID'] = df_class.apply(clean_class_id, axis=1)
df_class['CLASS-ID'] = df_class['CLASS-ID'].astype(int)

In [ ]:
df_test1nn = pd.merge(df_test1nn_report, df_class[['CLASS-ID', 'TRANSFORM_NAME', 'TRANSFORM_ID']], on='CLASS-ID', how='left')

In [ ]:
df_test1nn

In [ ]:
df_test1nn.to_csv('data/OptBertFPFinal2/df_test1nn.csv', index=False)

In [ ]:
# Calculate macro average for precision, recall, and f1-score
macro_precision2 = df_test1nn['precision'].mean()
macro_recall2 = df_test1nn['recall'].mean()
macro_f12 = df_test1nn['f1-score'].mean()

# Calculate weighted average for precision, recall, and f1-score
weighted_precision2 = (df_test1nn['precision'] * df_test1nn['test_support']).sum() / df_test1nn['test_support'].sum()
weighted_recall2 = (df_test1nn['recall'] * df_test1nn['test_support']).sum() / df_test1nn['test_support'].sum()
weighted_f12 = (df_test1nn['f1-score'] * df_test1nn['test_support']).sum() / df_test1nn['test_support'].sum()

In [ ]:
output1 = f"""
Macro-averages:
- Precision: {macro_precision2:.4f}
- Recall: {macro_recall2:.4f}
- F1-score: {macro_f12:.4f}

Weighted-averages:
- Precision: {weighted_precision2:.4f}
- Recall: {weighted_recall2:.4f}
- F1-score: {weighted_f12:.4f}
"""

In [ ]:
print(output1)